In [ ]:
import os
import numpy as np
from gvec_to_python.reader.gvec_reader import create_GVEC_json
from gvec_to_python import GVEC
import gvec_to_python as _
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import axes3d

# give absolute paths to the files

pkg_path = _.__path__[0]
print('package path:', pkg_path)
#gvec_file='testcases/ellipstell/newBC_E1D6_M6N6/GVEC_ELLIPSTELL_E1D6_M6N6_State_0000_00200000'
gvec_file='testcases/ellipstell/newBC_E4D6_M6N6/GVEC_ELLIPSTELL_E4D6_M6N6_State_0001_00200000'

dat_file_in   = os.path.join(pkg_path, gvec_file+'.dat')
json_file_out = os.path.join(pkg_path, gvec_file+'.json')

create_GVEC_json(dat_file_in, json_file_out)

# main object (one without, the other one with pyccel kernels)
gvec = GVEC(json_file_out,unit_tor_domain="one-fp",use_pyccel=True)

# Plot profiles

In [ ]:
s = np.linspace(0, 1, 100)

fig = plt.figure(figsize=(13, 10))

# toroidal flux
ax = fig.add_subplot(2, 2, 1)
ax.plot(s, gvec.profiles.profile(s, name='phi'))
ax.set_xlabel('s')
ax.set_ylabel('$\Phi(s)$')
ax.set_title('Toroidal flux')

# poloidal flux
ax = fig.add_subplot(2, 2, 2)
ax.plot(s, gvec.profiles.profile(s, name='chi'))
ax.set_xlabel('s')
ax.set_ylabel('$\chi(s)$')
ax.set_title('Poloidal flux')

# iota
ax = fig.add_subplot(2, 2, 3)
ax.plot(s, gvec.profiles.profile(s, name='iota'))
ax.set_xlabel('s')
ax.set_ylabel('$\iota(s)$')
ax.set_title('Iota')

# pressure
ax = fig.add_subplot(2, 2, 4)
ax.plot(s, gvec.profiles.profile(s, name='pressure'))
ax.set_xlabel('s')
ax.set_ylabel('$p(s)$')
ax.set_title('Pressure')

# Poloidal geometry

In [ ]:

def get_uvfac(mapping):
    '''Depending on the mapping, gives back the factor for u in [0,1] (either 1 or 2pi) 
       and v in[0,1] scaled to always represent one field period'''
    if(mapping=='unit' or mapping=='unit_pest'):
        ufac=1.
        # 'unit' mapping can include a tor_fraction. We want to always visualize one field period.
        vfac = 1./(gvec.domain.tor_fraction*gvec.nfp)
    else: #'gvec',"wo_hmap",'pest'
        ufac=2*np.pi
        vfac=2*np.pi/gvec.nfp
    return ufac,vfac


In [ ]:
gvec.mapping = 'gvec' # use the mapping setter

n_planes = 4 
s = np.linspace(0, 1, 20) # radial coordinate in [0, 1]
ufac,vfac=get_uvfac(gvec.mapping) # compute factors to always visualize one field period
a1 = np.linspace(0, 1*ufac, 39) # poloidal angle in [0, 1]
a2 = np.linspace(0, 1*vfac, n_planes,endpoint=False) 

x, y, z = gvec.f(s, a1, a2)

In [ ]:
fig = plt.figure(figsize=(13, np.ceil(n_planes/2) * 6.5))

# poloidal plane grid
for n in range(a2.size):
    if(gvec.mapping=="wo_hmap"):
        rp = x[:, :, n].squeeze()
        zp = y[:, :, n].squeeze()
    else:
        xp = x[:, :, n].squeeze()
        yp = y[:, :, n].squeeze()
        zp = z[:, :, n].squeeze()

        rp = np.sqrt(xp**2 + yp**2)
    

    ax = fig.add_subplot(int(np.ceil(n_planes/2)), 2, n + 1)
    for i in range(rp.shape[0]):
        for j in range(rp.shape[1] - 1):
            if i < rp.shape[0] - 1:
                ax.plot([rp[i, j], rp[i + 1, j]], [zp[i, j], zp[i + 1, j]], 'b', linewidth=.6)
            if j < rp.shape[1] - 1:
                ax.plot([rp[i, j], rp[i, j + 1]], [zp[i, j], zp[i, j + 1]], 'b', linewidth=.6)
    ax.set_xlabel('R')
    ax.set_ylabel('Z')
    ax.axis('equal')
    ax.set_title('Poloidal plane at $\\zeta$={0:4.3f} $\pi$'.format(a2[n]/vfac*2/gvec.nfp))

# Jacobian determinant

In [ ]:
gvec.mapping = 'gvec' # use the mapping setter

n_planes = 4 
s = np.linspace(0, 1, 20) # radial coordinate in [0, 1]
ufac,vfac=get_uvfac(gvec.mapping) # compute factors to always visualize one field period
u = np.linspace(0, 1*ufac, 49) # poloidal angle 
v = np.linspace(0, 1*vfac, n_planes,endpoint=False) #toroidal angle

x, y, z = gvec.f(s, u, v)
det_df  = gvec.det_df(s, u, v)

In [ ]:
fig = plt.figure(figsize=(13, np.ceil(n_planes/2) * 6.5))

# poloidal plane grid
for n in range(v.size):

    xp = x[:, :, n].squeeze()
    yp = y[:, :, n].squeeze()
    zp = z[:, :, n].squeeze()

    rp = np.sqrt(xp**2 + yp**2)

    detp = det_df[:, :, n].squeeze()

    ax = fig.add_subplot(int(np.ceil(n_planes/2)), 2, n + 1)
    map = ax.contourf(rp, zp, detp, 30)
    ax.set_xlabel('R')
    ax.set_ylabel('Z')
    ax.axis('equal')
    ax.set_title('Jacobian determinant at $\\zeta$={0:4.3f} $\pi$'.format(v[n]/vfac*2/gvec.nfp))
    fig.colorbar(map, ax=ax, location='right')

# Top view

In [ ]:
gvec.mapping = 'unit' # use the mapping setter

s = np.linspace(0, 1, 20) # radial coordinate in [0, 1]
u = np.linspace(0, 1, 3) # poloidal angle in [0, 1]
v = np.linspace(0, 1, 69) # toroidal angle in [0, 1]

x, y, z = gvec.f(s, u, v)

In [ ]:
fig = plt.figure(figsize=(13, 2 * 6.5))
ax = fig.add_subplot()

# poloidal plane grid
for m in range(2):

    xp = x[:, m, :].squeeze()
    yp = y[:, m, :].squeeze()
    zp = z[:, m, :].squeeze()

    for i in range(xp.shape[0]):
        for j in range(xp.shape[1] - 1):
            if i < xp.shape[0] - 1:
                ax.plot([xp[i, j], xp[i + 1, j]], [yp[i, j], yp[i + 1, j]], 'b', linewidth=.6)
            if j < xp.shape[1] - 1:
                if i == 0:
                    ax.plot([xp[i, j], xp[i, j + 1]], [yp[i, j], yp[i, j + 1]], 'r', linewidth=1)
                else:
                    ax.plot([xp[i, j], xp[i, j + 1]], [yp[i, j], yp[i, j + 1]], 'b', linewidth=.6)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.axis('equal')
    ax.set_title('Stellarator top view')

# Pressure

In [ ]:
gvec.mapping = 'unit' # use the mapping setter

n_planes = 4
s = np.linspace(0, 1, 20) # radial coordinate in [0, 1]
ufac,vfac=get_uvfac(gvec.mapping) # compute factors to always visualize one field period
u = np.linspace(0, 1*ufac, 49) # poloidal angle 
v = np.linspace(0, 1*vfac, n_planes,endpoint=False) #toroidal angle

p, xyz = gvec.p_cart(s, u, v)

In [ ]:
fig = plt.figure(figsize=(15, np.ceil(n_planes/2) * 6.5))

# poloidal plane grid
for n in range(v.size):

    xp = xyz[0][:, :, n].squeeze()
    yp = xyz[1][:, :, n].squeeze()
    zp = xyz[2][:, :, n].squeeze()

    rp = np.sqrt(xp**2 + yp**2)

    pp = p[:, :, n].squeeze()

    ax = fig.add_subplot(int(np.ceil(n_planes/2)), 2, n + 1)
    map = ax.contourf(rp, zp, pp, 30)
    ax.set_xlabel('R')
    ax.set_ylabel('Z')
    ax.axis('equal')
    ax.set_title('Pressure at $\\zeta$={0:4.3f} $\pi$'.format(v[n]/vfac * 2 / gvec.nfp))
    fig.colorbar(map, ax=ax, location='right')

# Magnetic field

In [ ]:
gvec.mapping = 'gvec' # use the mapping setter

n_planes = 4
s = np.linspace(0.0001, 1, 13) # radial coordinate in [0, 1]
ufac,vfac=get_uvfac(gvec.mapping) # compute factors to always visualize one field period
u = np.linspace(0, 1*ufac, 49) # poloidal angle 
v = np.linspace(0, 1*vfac, n_planes,endpoint=False) #toroidal angle

b, xyz = gvec.b_cart(s, u, v)

df = gvec.df(s,u,v)
if(gvec.mapping=="wo_hmap"):
    abs_b = np.sqrt(b[0]**2 + b[1]**2 + xyz[0]**2*b[2]**2)
else:
    abs_b = np.sqrt(b[0]**2 + b[1]**2 + b[2]**2)

print(abs_b.shape)

In [ ]:
fig = plt.figure(figsize=(15, np.ceil(n_planes/2) * 6.5))

for n in range(v.size):

    if(gvec.mapping=="wo_hmap"):
        rp = xyz[0][:, :, n].squeeze()
        zp = xyz[1][:, :, n].squeeze()
    else:
        xp = xyz[0][:, :, n].squeeze()
        yp = xyz[1][:, :, n].squeeze()
        zp = xyz[2][:, :, n].squeeze()

        rp = np.sqrt(xp**2 + yp**2)

    ab = abs_b[:, :, n].squeeze()

    ax = fig.add_subplot(int(np.ceil(n_planes/2)), 2, n + 1)
    map = ax.contourf(rp, zp, ab, 30)
    ax.set_xlabel('R')
    ax.set_ylabel('Z')
    ax.axis('equal')
    ax.set_title('Magnetic field strength at $\\zeta$={0:4.3f} $\pi$'.format(v[n]/vfac * 2 / gvec.nfp))
    fig.colorbar(map, ax=ax, location='right')

In [ ]:
fig = plt.figure(figsize=(13, np.ceil(n_planes/2) * 6.5))

for n in range(v.size):


    xp = xyz[0][:, :, n].squeeze()
    yp = xyz[1][:, :, n].squeeze()
    zp = xyz[2][:, :, n].squeeze()

    # plot theta derivative of mapping, 
    dxdu = df[:, :, n,0,1].squeeze()
    dydu = df[:, :, n,1,1].squeeze()
    dzdu = df[:, :, n,2,1].squeeze()

    zeta = 2*np.pi/ gvec.nfp *v[n]/vfac

    drdu = dxdu*np.cos(zeta) - dydu*np.sin(zeta)



    ax = fig.add_subplot(int(np.ceil(n_planes/2)), 2, n + 1)
    ax.quiver(rp, zp, drdu, dzdu, scale=20)
    ax.set_xlabel('R')
    ax.set_ylabel('Z')
    ax.axis('equal')
    ax.set_title('Theta derivative of mapping at $\\zeta$={0:4.3f} $\pi$'.format(v[n] * 2 / gvec.domain.nfp))
    #fig.colorbar(map, ax=ax, location='right')

# Current density

In [ ]:
def bcart(s,th,ze):
    b,x=gvec.b_cart(s,th,ze)
    return b

def jcart_FD(s,th,ze):
    eps=1.0e-8
    b_cart = bcart(s,th,ze)
    b_cart_s_eps   = bcart(s+eps,th,ze)
    b_cart_th_eps  = bcart(s,th+eps,ze)
    b_cart_ze_eps  = bcart(s,th,ze+eps)
    gradb=np.zeros((3,3))
    for d in [0,1,2]:
        b_ds =(b_cart_s_eps[d] -b_cart[d])/eps
        b_dth=(b_cart_th_eps[d]-b_cart[d])/eps
        b_dze=(b_cart_ze_eps[d]-b_cart[d])/eps
        gradb[d] = gvec.df_inv(s,th,ze).T@np.array([b_ds,b_dth,b_dze])
    j_cart_FD=np.zeros(3)
    for d in [0,1,2]:
        j_cart_FD[d] = gradb[(d+2)%3][(d+1)%3]-gradb[(d+1)%3][(d+2)%3]
    return j_cart_FD

gvec.mapping = 'gvec'
s,th,ze = [0.49,0.5,0.3]

mu0=4.0e-7*np.pi
j_ex ,xyz = gvec.j_cart(s, th, ze)
j_FD = jcart_FD(s,th,ze)/mu0
print('j_FD ',j_FD)
print('j_ex',np.array(j_ex))
print('j_ex-j_FD',np.array(j_ex)-j_FD)
assert(np.allclose(j_ex,j_FD,rtol=1e-6,atol=1e-6/mu0))

In [ ]:
gvec.mapping = 'gvec'

n_planes = 4
s = np.linspace(0.001, 1, 10) # radial coordinate in [0, 1]
ufac,vfac=get_uvfac(gvec.mapping) # compute factors to always visualize one field period
u = np.linspace(0, 1*ufac, 49) # poloidal angle 
v = np.linspace(0, 1*vfac, n_planes,endpoint=False) #toroidal angle

j, xyz = gvec.j_cart(s, u, v)

abs_j = np.sqrt(j[0]**2 + j[1]**2 + j[2]**2)

In [ ]:
fig = plt.figure(figsize=(15, np.ceil(n_planes/2) * 6.5))

for n in range(v.size):

    xp = xyz[0][:, :, n].squeeze()
    yp = xyz[1][:, :, n].squeeze()
    zp = xyz[2][:, :, n].squeeze()

    rp = np.sqrt(xp**2 + yp**2)

    ab = abs_j[:, :, n].squeeze()

    ax = fig.add_subplot(int(np.ceil(n_planes/2)), 2, n + 1)
    map = ax.contourf(rp, zp, ab, 30)
    ax.set_xlabel('R')
    ax.set_ylabel('Z')
    ax.axis('equal')
    ax.set_title('Current (absolute value) at $\\zeta$={0:4.3f} $\pi$'.format(v[n]/vfac * 2 / gvec.nfp))
    fig.colorbar(map, ax=ax, location='right')

In [ ]:
gvec.assert_equil(s, u, v)

In [ ]:
p = gvec.profiles.profile(s, u, v, name='pressure', der=None) 
p_s = gvec.profiles.profile(s, u, v, name='pressure', der='s') 
j = gvec.j2(s, u, v)
b = gvec.bv(s, u, v)
abs_j = np.sqrt(j[0]**2 + j[1]**2 + j[2]**2)
abs_b = np.sqrt(b[0]**2 + b[1]**2 + b[2]**2)

In [ ]:
fig = plt.figure(figsize=(15, np.ceil(n_planes/2) * 6.5))

ss, uu, vv = gvec.domain.prepare_args(s, u, v, sparse=False)

print(ss.shape, uu.shape, vv.shape)

comp = 0

for n in range(v.size):

    sp = ss[:, :, n].squeeze()
    up = uu[:, :, n].squeeze()
    vp = vv[:, :, n].squeeze()

    ji = j[comp][:, :, n].squeeze()

    ax = fig.add_subplot(int(np.ceil(n_planes/2)), 2, n + 1)
    map = ax.contourf(sp, up, ji, 30)
    ax.set_xlabel('s')
    ax.set_ylabel('u')
    ax.set_title('Current component {1} at $\\phi$={0:4.1f} $\pi$'.format(v[n] * 2 / gvec.domain.nfp, comp))
    fig.colorbar(map, ax=ax, location='right')

In [ ]:
fig = plt.figure(figsize=(15, np.ceil(n_planes/2) * 6.5))

ss, uu, vv = gvec.domain.prepare_args(s, u, v, sparse=False)

print(ss.shape, uu.shape, vv.shape)

comp = 0

for n in range(v.size):

    sp = ss[:, :, n].squeeze()
    up = uu[:, :, n].squeeze()
    vp = vv[:, :, n].squeeze()

    if(comp==0):
        fbe = (p_s - (j[1] * b[2] - j[2] * b[1]))[:, :, n].squeeze()
    elif(comp==1):
        fbe = (j[2] * b[0] - j[0] * b[2])[:, :, n].squeeze()
    elif(comp==2):
        fbe = (j[0] * b[1] - j[1] * b[0])[:, :, n].squeeze()
    ax = fig.add_subplot(int(np.ceil(n_planes/2)), 2, n + 1)
    map = ax.contourf(sp, up, fbe, 30)
    ax.set_xlabel('s')
    ax.set_ylabel('u')
    ax.set_title('force balance error in comp {1} at $\\zeta$={0:4.1f} $\pi$'.format(v[n]/vfac * 2 / gvec.nfp, comp))
    fig.colorbar(map, ax=ax, location='right')